In [11]:
import logging
import re
from datetime import datetime
from langdetect.lang_detect_exception import LangDetectException
from pymongo.errors import BulkWriteError
from config import mongodb
import pandas as pd
from langdetect import detect

logger = logging.getLogger()
logger.setLevel(logging.WARNING)

In [12]:
client = mongodb()
scrapperdb = client['scrapper']
collection = scrapperdb['tweets']


In [14]:
reader = pd.read_csv('../../data/kaggle/bitcoin-tweets-20160101-to-20190329/tweets.csv', engine='python', sep=';', chunksize=5000)
for j, chunk in enumerate(reader):
    if j < 681:
        continue
    buffer = []
    for i, row in chunk.iterrows():
        try:
            if not row['text'] or not isinstance(row['text'], str) or detect(row['text']) != 'en':
                continue
        except LangDetectException as e:
            continue

        result = {
            '_id': row['id'],
            'text': row['text'],
            'user_id': row['user'],
            'follower_count': None,
            'verified': None,
            'created_at': datetime.strptime(row['timestamp']+'00', '%Y-%m-%d %H:%M:%S%z'),
            'retweet_count': row['retweets'],
            'favorite_count': row['likes'],
            'topics': [
                tag.lower()
                for tag in re.findall(r"#(\w*[0-9a-zA-Z]+\w*[0-9a-zA-Z])", row['text'])
            ]
        }
        buffer.append(result)

    if len(buffer) == 0:
        print(f'Skipped empty: {j}')
        continue
    try:
        result = collection.insert_many(buffer, ordered=False)
        print(f'Inserted: {len(result.inserted_ids)}')
    except BulkWriteError as e:
        print(f'Skipped something: {j}')
        pass

Skipped something: 681
Skipped something: 682
Skipped something: 683
Inserted: 3833
Skipped something: 685
Inserted: 4177
Skipped something: 687
Inserted: 4044
Skipped something: 689
Inserted: 3999
Skipped something: 691
Skipped something: 692
Inserted: 3983
Skipped something: 694
Skipped something: 695
Skipped something: 696
Skipped something: 697
Skipped something: 698
Skipped something: 699
Skipped something: 700
Inserted: 3916
Skipped something: 702
Inserted: 4061
Inserted: 4003
Skipped something: 705
Inserted: 3913
Inserted: 3872
Skipped something: 708
Skipped something: 709
Inserted: 3679
Skipped something: 711
Inserted: 3902
Skipped something: 713
Skipped something: 714
Skipped something: 715
Skipped something: 716
Skipped something: 717
Skipped something: 718
Skipped something: 719
Inserted: 3753
Inserted: 3741
Skipped something: 722
Skipped something: 723
Skipped something: 724
Skipped something: 725
Skipped something: 726
Skipped something: 727
Skipped something: 728
Skipped 

KeyboardInterrupt: 